In [ ]:
%%configure -f 
{"driverMemory": "6000M"}

Airline Satisfaction Survey Classification Model

In [ ]:
from pyspark.ml import Pipeline
import pyspark.ml.feature as feat
import pyspark.ml.tuning as tune
import pyspark.ml.classification as cl
import pyspark.ml.evaluation as ev

VBox()

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
1,application_1638168656605_0002,pyspark,idle,Link,Link,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [ ]:
airline_csv = (spark.read.csv('s3://aws-logs-726097832390-us-east-1/Spark/satisfaction.csv', 
                                  header=True, inferSchema=True))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

# Preprocessing and Transformations

Here, we compare the count vs the distinct count to determine if the dataset has any duplicates.

In [ ]:
airline_csv.count(), airline_csv.distinct().count()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

(129880, 129880)

As shown above, there are no duplicates in the dataset.

Next, we will check to see if there are null vales in any of the columns.

In [ ]:
from pyspark.sql.functions import isnan, when, count, col
airline_csv.select([count(when(isnan(c), c)).alias(c) for c in airline_csv.columns]).show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+---+---------------+------+-------------+---+--------------+-----+---------------+------------+---------------------------------+--------------+-------------+---------------------+----------------------+--------------+----------------------+----------------+----------------+----------------+---------------+-----------+---------------+--------------------------+------------------------+
| id|satisfaction_v2|Gender|Customer Type|Age|Type of Travel|Class|Flight Distance|Seat comfort|Departure/Arrival time convenient|Food and drink|Gate location|Inflight wifi service|Inflight entertainment|Online support|Ease of Online booking|On-board service|Leg room service|Baggage handling|Checkin service|Cleanliness|Online boarding|Departure Delay in Minutes|Arrival Delay in Minutes|
+---+---------------+------+-------------+---+--------------+-----+---------------+------------+---------------------------------+--------------+-------------+---------------------+----------------------+--------------+-

According to the chart, there are no null values in any column. However, we will use .dropna() just to be sure.

In [ ]:
airline_csv = airline_csv.dropna()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Next, we must rename all columns so that there are no spaces in any column names.

In [ ]:
airline_csv.printSchema()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- id: integer (nullable = true)
 |-- satisfaction_v2: string (nullable = true)
 |-- Gender: string (nullable = true)
 |-- Customer Type: string (nullable = true)
 |-- Age: integer (nullable = true)
 |-- Type of Travel: string (nullable = true)
 |-- Class: string (nullable = true)
 |-- Flight Distance: integer (nullable = true)
 |-- Seat comfort: integer (nullable = true)
 |-- Departure/Arrival time convenient: integer (nullable = true)
 |-- Food and drink: integer (nullable = true)
 |-- Gate location: integer (nullable = true)
 |-- Inflight wifi service: integer (nullable = true)
 |-- Inflight entertainment: integer (nullable = true)
 |-- Online support: integer (nullable = true)
 |-- Ease of Online booking: integer (nullable = true)
 |-- On-board service: integer (nullable = true)
 |-- Leg room service: integer (nullable = true)
 |-- Baggage handling: integer (nullable = true)
 |-- Checkin service: integer (nullable = true)
 |-- Cleanliness: integer (nullable = true)
 |-- Onli

In [ ]:
re_airline_csv = airline_csv.withColumnRenamed("satisfaction_v2", "satisfaction") \
.withColumnRenamed("Gender", "gender") \
.withColumnRenamed("Customer Type", "customer_type") \
.withColumnRenamed("Age", "age") \
.withColumnRenamed("Type of Travel", "type_of_travel") \
.withColumnRenamed("Class", "class") \
.withColumnRenamed("Flight Distance", "flight_distance") \
.withColumnRenamed("Seat Comfort", "seat_comfort") \
.withColumnRenamed("Departure/Arrival time convenient", "departure_arrival_time_convenient") \
.withColumnRenamed("Food and Drink", "food_and_drink") \
.withColumnRenamed("Gate location", "gate_location") \
.withColumnRenamed("Inflight wifi service", "inflight_wifi_service") \
.withColumnRenamed("Inflight entertainment", "inflight_entertainment") \
.withColumnRenamed("Online support", "online_support") \
.withColumnRenamed("Ease of Online booking", "ease_of_online_booking") \
.withColumnRenamed("On-board service", "on_board_service") \
.withColumnRenamed("Leg room service", "leg_room") \
.withColumnRenamed("Baggage handling", "baggage_handling") \
.withColumnRenamed("Checkin service", "checkin_service") \
.withColumnRenamed("Cleanliness", "cleanliness") \
.withColumnRenamed("Online boarding", "online_boarding") \
.withColumnRenamed("Departure Delay in Minutes", "departure_delay_minutes") \
.withColumnRenamed("Arrival Delay in Minutes", "arrival_delay_minutes")

re_airline_csv.printSchema()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- id: integer (nullable = true)
 |-- satisfaction: string (nullable = true)
 |-- gender: string (nullable = true)
 |-- customer_type: string (nullable = true)
 |-- age: integer (nullable = true)
 |-- type_of_travel: string (nullable = true)
 |-- class: string (nullable = true)
 |-- flight_distance: integer (nullable = true)
 |-- seat_comfort: integer (nullable = true)
 |-- departure_arrival_time_convenient: integer (nullable = true)
 |-- food_and_drink: integer (nullable = true)
 |-- gate_location: integer (nullable = true)
 |-- inflight_wifi_service: integer (nullable = true)
 |-- inflight_entertainment: integer (nullable = true)
 |-- online_support: integer (nullable = true)
 |-- ease_of_online_booking: integer (nullable = true)
 |-- on_board_service: integer (nullable = true)
 |-- leg_room: integer (nullable = true)
 |-- baggage_handling: integer (nullable = true)
 |-- checkin_service: integer (nullable = true)
 |-- cleanliness: integer (nullable = true)
 |-- online_boarding

We have successfully renamed all column names so that we have no spaces.

## Creating Dummies

Before continuing, we have to convert all string columns to numeric values using dummies. Specifically, the columns we will be changing are: satisfaction, gender, customer_type, type_of_travel, and class.

In [ ]:
import pyspark.sql.functions as f
categorical_columns=["gender", "customer_type", "type_of_travel", "class", "satisfaction"]
expressions=[]
for column in categorical_columns:
    expression = re_airline_csv.select(column).distinct().rdd.flatMap(lambda x: x).collect()
    types_expr = [f.when(f.col(column) == ty, 1).otherwise(0).alias("e_{}_{}".format(column, ty)) for ty in expression]
    expressions += types_expr
dum_airline_csv = re_airline_csv.select("id", "age", "flight_distance", "seat_comfort", "departure_arrival_time_convenient",
                                        "food_and_drink", "gate_location", "inflight_wifi_service", "inflight_entertainment",
                                        "online_support", "ease_of_online_booking", "on_board_service", "leg_room",
                                        "baggage_handling", "checkin_service", "cleanliness", "online_boarding",
                                        "departure_delay_minutes", "arrival_delay_minutes", *categorical_columns, *expressions)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [ ]:
dum_airline_csv.show(3)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+------+---+---------------+------------+---------------------------------+--------------+-------------+---------------------+----------------------+--------------+----------------------+----------------+--------+----------------+---------------+-----------+---------------+-----------------------+---------------------+------+--------------+---------------+--------+------------+-------------+---------------+------------------------------+---------------------------------+--------------------------------+--------------------------------+----------------+----------------+-----------+--------------------------------------+------------------------+
|    id|age|flight_distance|seat_comfort|departure_arrival_time_convenient|food_and_drink|gate_location|inflight_wifi_service|inflight_entertainment|online_support|ease_of_online_booking|on_board_service|leg_room|baggage_handling|checkin_service|cleanliness|online_boarding|departure_delay_minutes|arrival_delay_minutes|gender| customer_type| type_

Now, we have to drop the unnecessary columns for gender, satisfaction, customer_type, and type_of_travel. This is because there are only two categories for these columns and it is unnecessary having more than one column. We will also rename the columns for better readability.

In [ ]:
dum_airline_csv.printSchema()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- id: integer (nullable = true)
 |-- age: integer (nullable = true)
 |-- flight_distance: integer (nullable = true)
 |-- seat_comfort: integer (nullable = true)
 |-- departure_arrival_time_convenient: integer (nullable = true)
 |-- food_and_drink: integer (nullable = true)
 |-- gate_location: integer (nullable = true)
 |-- inflight_wifi_service: integer (nullable = true)
 |-- inflight_entertainment: integer (nullable = true)
 |-- online_support: integer (nullable = true)
 |-- ease_of_online_booking: integer (nullable = true)
 |-- on_board_service: integer (nullable = true)
 |-- leg_room: integer (nullable = true)
 |-- baggage_handling: integer (nullable = true)
 |-- checkin_service: integer (nullable = true)
 |-- cleanliness: integer (nullable = true)
 |-- online_boarding: integer (nullable = true)
 |-- departure_delay_minutes: integer (nullable = true)
 |-- arrival_delay_minutes: integer (nullable = true)
 |-- gender: string (nullable = true)
 |-- customer_type: string (nullab

In [ ]:
columns_to_drop = ("satisfaction", "gender", "customer_type", "type_of_travel", "class", 
                   "e_satisfaction_neutral or dissatisfied", "e_gender_male", "e_customer_type_disloyal Customer",
                  "e_type_of_travel_Business travel")
dum_airline_csv = dum_airline_csv.drop(*columns_to_drop)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [ ]:
dum_airline_csv.printSchema()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- id: integer (nullable = true)
 |-- age: integer (nullable = true)
 |-- flight_distance: integer (nullable = true)
 |-- seat_comfort: integer (nullable = true)
 |-- departure_arrival_time_convenient: integer (nullable = true)
 |-- food_and_drink: integer (nullable = true)
 |-- gate_location: integer (nullable = true)
 |-- inflight_wifi_service: integer (nullable = true)
 |-- inflight_entertainment: integer (nullable = true)
 |-- online_support: integer (nullable = true)
 |-- ease_of_online_booking: integer (nullable = true)
 |-- on_board_service: integer (nullable = true)
 |-- leg_room: integer (nullable = true)
 |-- baggage_handling: integer (nullable = true)
 |-- checkin_service: integer (nullable = true)
 |-- cleanliness: integer (nullable = true)
 |-- online_boarding: integer (nullable = true)
 |-- departure_delay_minutes: integer (nullable = true)
 |-- arrival_delay_minutes: integer (nullable = true)
 |-- e_gender_Female: integer (nullable = false)
 |-- e_customer_type_Lo

In [ ]:
dum_airline_csv = dum_airline_csv.withColumnRenamed("e_satisfaction_satisfied", "satisfied") \
.withColumnRenamed("e_gender_Female", "female") \
.withColumnRenamed("e_customer_type_Loyal Customer", "loyal_customer") \
.withColumnRenamed("e_type_of_travel_Personal Travel", "personal_travel") \
.withColumnRenamed("e_class_Eco Plus", "eco_plus") \
.withColumnRenamed("e_class_Business", "business") \
.withColumnRenamed("e_class_Eco", "eco")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [ ]:
dum_airline_csv.printSchema()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- id: integer (nullable = true)
 |-- age: integer (nullable = true)
 |-- flight_distance: integer (nullable = true)
 |-- seat_comfort: integer (nullable = true)
 |-- departure_arrival_time_convenient: integer (nullable = true)
 |-- food_and_drink: integer (nullable = true)
 |-- gate_location: integer (nullable = true)
 |-- inflight_wifi_service: integer (nullable = true)
 |-- inflight_entertainment: integer (nullable = true)
 |-- online_support: integer (nullable = true)
 |-- ease_of_online_booking: integer (nullable = true)
 |-- on_board_service: integer (nullable = true)
 |-- leg_room: integer (nullable = true)
 |-- baggage_handling: integer (nullable = true)
 |-- checkin_service: integer (nullable = true)
 |-- cleanliness: integer (nullable = true)
 |-- online_boarding: integer (nullable = true)
 |-- departure_delay_minutes: integer (nullable = true)
 |-- arrival_delay_minutes: integer (nullable = true)
 |-- female: integer (nullable = false)
 |-- loyal_customer: integer (nu

In [ ]:
dum_airline_csv.show(5)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+------+---+---------------+------------+---------------------------------+--------------+-------------+---------------------+----------------------+--------------+----------------------+----------------+--------+----------------+---------------+-----------+---------------+-----------------------+---------------------+------+--------------+---------------+--------+--------+---+---------+
|    id|age|flight_distance|seat_comfort|departure_arrival_time_convenient|food_and_drink|gate_location|inflight_wifi_service|inflight_entertainment|online_support|ease_of_online_booking|on_board_service|leg_room|baggage_handling|checkin_service|cleanliness|online_boarding|departure_delay_minutes|arrival_delay_minutes|female|loyal_customer|personal_travel|eco_plus|business|eco|satisfied|
+------+---+---------------+------------+---------------------------------+--------------+-------------+---------------------+----------------------+--------------+----------------------+----------------+--------+-----

Our data is now transformed and we are ready to check correlations and create a pipeline and model. We created dummies for 5 columns. 4 columns were "n-1" and 1 column was "n" dummies.

# Correlations and Interpretations

Here, we will create correlations tables to see what columns are correlated with each other.

In [ ]:
features = ["satisfied", "age", "flight_distance", "female", "loyal_customer"]
n_features = len(features)

corr = []

for i in range(0, n_features):
    temp = [None] * i

    for j in range(i, n_features):
        temp.append(dum_airline_csv.corr(features[i], features[j]))
    corr.append([features[i]] + temp)

correlations = spark.createDataFrame(corr, ['Column'] + features)

correlations.show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+---------------+---------+-------------------+--------------------+--------------------+--------------------+
|         Column|satisfied|                age|     flight_distance|              female|      loyal_customer|
+---------------+---------+-------------------+--------------------+--------------------+--------------------+
|      satisfied|      1.0|0.11791263646280749|-0.03913285288944289| 0.21205024349542503|  0.2925625580192301|
|            age|     null|                1.0|-0.24943817760589548|-0.00898365375217...|  0.2842753859512764|
|flight_distance|     null|               null|                 1.0|-0.12075476065714706|-0.01904370686465537|
|         female|     null|               null|                null|                 1.0|-0.03080345215123...|
| loyal_customer|     null|               null|                null|                null|                 1.0|
+---------------+---------+-------------------+--------------------+--------------------+--------------------+

By the looks of this table, satisfaction is not very correlated at all with age, flight distance, gender, or customer loyalty. Let's try another table with some different columns.

In [ ]:
features = ["satisfied", "seat_comfort", "food_and_drink", "inflight_wifi_service", "inflight_entertainment"]
n_features = len(features)

corr = []

for i in range(0, n_features):
    temp = [None] * i

    for j in range(i, n_features):
        temp.append(dum_airline_csv.corr(features[i], features[j]))
    corr.append([features[i]] + temp)

correlations = spark.createDataFrame(corr, ['Column'] + features)

correlations.show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------------------+---------+-------------------+-------------------+---------------------+----------------------+
|              Column|satisfied|       seat_comfort|     food_and_drink|inflight_wifi_service|inflight_entertainment|
+--------------------+---------+-------------------+-------------------+---------------------+----------------------+
|           satisfied|      1.0|0.24237090064924635|0.12056826384376021|  0.22700974166867785|    0.5233635360253334|
|        seat_comfort|     null|                1.0| 0.7160076670175409|  0.12915196111305233|    0.4264879448156278|
|      food_and_drink|     null|               null|                1.0| 0.026076740181571335|    0.3677768909458935|
|inflight_wifi_ser...|     null|               null|               null|                  1.0|    0.2539405809633243|
|inflight_entertai...|     null|               null|               null|                 null|                   1.0|
+--------------------+---------+-------------------+----

Nothing in this chart is highly correlated either. We have a slightly higher correlation with inflight entertainment and satisfaction, but nothing too significant. Let's do another chart.

In [ ]:
features = ["satisfied", "departure_delay_minutes", "arrival_delay_minutes", "checkin_service", "baggage_handling"]
n_features = len(features)

corr = []

for i in range(0, n_features):
    temp = [None] * i

    for j in range(i, n_features):
        temp.append(dum_airline_csv.corr(features[i], features[j]))
    corr.append([features[i]] + temp)

correlations = spark.createDataFrame(corr, ['Column'] + features)

correlations.show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------------------+---------+-----------------------+---------------------+--------------------+--------------------+
|              Column|satisfied|departure_delay_minutes|arrival_delay_minutes|     checkin_service|    baggage_handling|
+--------------------+---------+-----------------------+---------------------+--------------------+--------------------+
|           satisfied|      1.0|    -0.0739598066163831| -0.08069079792257178|  0.2660889062029779|  0.2603982133177219|
|departure_delay_m...|     null|                    1.0|   0.9652911835463222|-0.02073443430462327|-0.01029693139684...|
|arrival_delay_min...|     null|                   null|                  1.0|-0.02385972574351...|-0.01416436335572...|
|     checkin_service|     null|                   null|                 null|                 1.0| 0.24120295959485144|
|    baggage_handling|     null|                   null|                 null|                null|                 1.0|
+--------------------+---------+

Correlation in this chart is not great across the columns, but it is clear that check-in service and baggage handling have stronger correlation to satisfaction than departure and arrival delays.

In [ ]:
features = ["satisfied", "cleanliness", "on_board_service", "leg_room", "gate_location"]
n_features = len(features)

corr = []

for i in range(0, n_features):
    temp = [None] * i

    for j in range(i, n_features):
        temp.append(dum_airline_csv.corr(features[i], features[j]))
    corr.append([features[i]] + temp)

correlations = spark.createDataFrame(corr, ['Column'] + features)

correlations.show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+----------------+---------+-------------------+------------------+-------------------+--------------------+
|          Column|satisfied|        cleanliness|  on_board_service|           leg_room|       gate_location|
+----------------+---------+-------------------+------------------+-------------------+--------------------+
|       satisfied|      1.0|0.25950405193992643|0.3522826069616563| 0.3051154393707849|-0.01227219686523...|
|     cleanliness|     null|                1.0|0.5511369213884671|0.41168152443982475|-0.00178932652854...|
|on_board_service|     null|               null|               1.0| 0.4118765989152811|-0.02534613054994...|
|        leg_room|     null|               null|              null|                1.0|-0.00746172126748...|
|   gate_location|     null|               null|              null|               null|                 1.0|
+----------------+---------+-------------------+------------------+-------------------+--------------------+

Lastly, we can see that cleanliness, on board service, and leg room have stronger correlation to satisfaction than gate location. Still, these numbers are not very high.

In [ ]:
column_order = ["id", "age", "flight_distance", "seat_comfort", "departure_arrival_time_convenient",
               "food_and_drink", "gate_location", "inflight_wifi_service", "inflight_entertainment",
               "online_support", "ease_of_online_booking", "on_board_service", "leg_room",
               "baggage_handling", "checkin_service", "cleanliness", "online_boarding", "departure_delay_minutes",
               "arrival_delay_minutes", "female", "loyal_customer", "personal_travel", "eco_plus",
               "business", "eco", "satisfied"]

ordered_airline = dum_airline_csv.select(column_order)
ordered_airline.printSchema()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- id: integer (nullable = true)
 |-- age: integer (nullable = true)
 |-- flight_distance: integer (nullable = true)
 |-- seat_comfort: integer (nullable = true)
 |-- departure_arrival_time_convenient: integer (nullable = true)
 |-- food_and_drink: integer (nullable = true)
 |-- gate_location: integer (nullable = true)
 |-- inflight_wifi_service: integer (nullable = true)
 |-- inflight_entertainment: integer (nullable = true)
 |-- online_support: integer (nullable = true)
 |-- ease_of_online_booking: integer (nullable = true)
 |-- on_board_service: integer (nullable = true)
 |-- leg_room: integer (nullable = true)
 |-- baggage_handling: integer (nullable = true)
 |-- checkin_service: integer (nullable = true)
 |-- cleanliness: integer (nullable = true)
 |-- online_boarding: integer (nullable = true)
 |-- departure_delay_minutes: integer (nullable = true)
 |-- arrival_delay_minutes: integer (nullable = true)
 |-- female: integer (nullable = false)
 |-- loyal_customer: integer (nu

The purpose of the code above is to change the order of columns in the DataFrame so that our dependent variable (satisfied) is in the last position.

# Chi-Square Selector

Here, we will create a vector assembler with all columns except the dependent variable. Then, we create a selector object. After, we put the vector assembler and the selector through the pipeline and fit and transform our data to the model. After that, we can display the top features in our dataset.

In [ ]:
vectorAssembler = feat.VectorAssembler(
    inputCols = ordered_airline.columns[1:-1]
    , outputCol = 'features'
)

selector = feat.ChiSqSelector(
    labelCol = 'satisfied'
    , numTopFeatures = 10
    , outputCol = 'selected')

pipeline_sel = Pipeline(stages=[vectorAssembler, selector])

model = (
    pipeline_sel
    .fit(ordered_airline)
    .transform(ordered_airline)
)

model.schema['selected'].metadata

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

{'ml_attr': {'attrs': {'numeric': [{'idx': 0, 'name': 'age'}, {'idx': 1, 'name': 'flight_distance'}, {'idx': 2, 'name': 'seat_comfort'}, {'idx': 3, 'name': 'departure_arrival_time_convenient'}, {'idx': 4, 'name': 'food_and_drink'}, {'idx': 5, 'name': 'gate_location'}, {'idx': 6, 'name': 'inflight_wifi_service'}, {'idx': 7, 'name': 'inflight_entertainment'}, {'idx': 8, 'name': 'online_support'}, {'idx': 9, 'name': 'ease_of_online_booking'}]}, 'num_attrs': 10}}

# Logistic Regression

First, we need to split our data into training and testing sets.

In [ ]:
airline_train, airline_test = (
    ordered_airline
    .randomSplit([0.7, 0.3], seed=666)
)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Next are steps to train the model using a pipeline.

In [ ]:
vectorAssembler = feat.VectorAssembler(
    inputCols = ordered_airline.columns[1:-1]
    , outputCol = 'features')

selector = feat.ChiSqSelector(
    labelCol = 'satisfied'
    , numTopFeatures = 5
    , outputCol = 'selected')

logReg_obj = cl.LogisticRegression(
    labelCol = 'satisfied'
    , featuresCol = selector.getOutputCol()
    , regParam = 0.01
    , elasticNetParam = 1.0
    , family = 'multinomial'
)

pipeline = Pipeline(
    stages=[
        vectorAssembler
        , selector
        , logReg_obj
    ])

pModel = pipeline.fit(airline_train)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Now, we will apply the model to the test set and evaluate the performance of the multinominal model in terms of F1 score, weighted percision, and accuracy. 

In [ ]:
logReg_model = (
    pModel
    .transform(airline_test)
)

results_logReg = logReg_model.select('satisfied', 'probability', 'prediction')

evaluator = ev.MulticlassClassificationEvaluator(
    predictionCol='prediction'
    , labelCol='satisfied')

(
    evaluator.evaluate(results_logReg)
    , evaluator.evaluate(
        results_logReg
        , {evaluator.metricName: 'weightedPrecision'}
    ) 
    , evaluator.evaluate(
        results_logReg
        , {evaluator.metricName: 'accuracy'}
    )
)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

(0.6321509051918819, 0.637113458340117, 0.6391000537510558)

So, for the multinominal logistic regression model, we have an F1 score of 63%, a weighted precision score of 64%, and an accuracy score of 64%. We can also print the selected features from the chisquare selector and the model coefficients.

In [ ]:
print(logReg_model.schema['selected'].metadata)

print(pModel.stages[-1].coefficientMatrix.toArray())

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

{'ml_attr': {'attrs': {'numeric': [{'idx': 0, 'name': 'age'}, {'idx': 1, 'name': 'flight_distance'}, {'idx': 2, 'name': 'seat_comfort'}, {'idx': 3, 'name': 'departure_arrival_time_convenient'}, {'idx': 4, 'name': 'food_and_drink'}]}, 'num_attrs': 5}}
[[-0.00709593  0.         -0.21223736  0.08857022  0.        ]
 [ 0.00709593  0.          0.21223736 -0.08857022  0.        ]]

# Random Forest Classifier

Next, we will run a random forest classifier model using a pipeline in the same manner we did for logistic regression. We will also apply the model to the test set and evaluate the performance of the random forest classifier in the same cell. We will measure the F1 score, the weighted precision, and the accuracy again.

In [ ]:
rf_obj = cl.RandomForestClassifier(
    labelCol = 'satisfied'
    , featuresCol = selector.getOutputCol()
    , minInstancesPerNode = 10
    , numTrees = 10
)

pipeline = Pipeline(
    stages = [vectorAssembler, selector, rf_obj]
)

pModel = pipeline.fit(airline_train)

rf_obj_trained = (
    pModel
    .transform(airline_test)
)
results_rf = rf_obj_trained.select('satisfied', 'probability', 'prediction')

evaluator = ev.MulticlassClassificationEvaluator(
    predictionCol = 'prediction'
    , labelCol = 'satisfied')

(
    evaluator.evaluate(results_rf)
    , evaluator.evaluate(
        results_rf
        , {evaluator.metricName: 'weightedPrecision'}
    )
    , evaluator.evaluate(
        results_rf
        , {evaluator.metricName: 'accuracy'}
    )
)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

(0.7542463926122396, 0.7563772306371901, 0.7561749724845785)

Here, we can see the random forest classifier has slightly better accuracy than the logistic regression model. The random forest classifier has an F1 score of 75.5%, a weighted precision score of 75.7%, and an accuracy score of 75.7%. We can also print the selected features from the chisquare selector.

In [ ]:
print(rf_obj_trained.schema['selected'].metadata)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

{'ml_attr': {'attrs': {'numeric': [{'idx': 0, 'name': 'age'}, {'idx': 1, 'name': 'flight_distance'}, {'idx': 2, 'name': 'seat_comfort'}, {'idx': 3, 'name': 'departure_arrival_time_convenient'}, {'idx': 4, 'name': 'food_and_drink'}]}, 'num_attrs': 5}}

From the looks of things, it seems the same features were selected for both models. 

# Hypertune Parameters with GridSearch

First, we will run GridSearch on the logistic regression model a few times to find the best parameters. GridSearch will make multiple models and find which parameters score the highest for the model. We will also use cross validator so that all records are able to be tested with each model. After, we will train the model with the best parameters.

In [ ]:
vectorAssembler = feat.VectorAssembler(
    inputCols = ordered_airline.columns[1:-1]
    , outputCol = 'features')

selector = feat.ChiSqSelector(
    labelCol = 'satisfied'
    , numTopFeatures = 5
    , outputCol = 'selected')

logReg_obj = cl.LogisticRegression(
    labelCol = 'satisfied'
    , featuresCol = selector.getOutputCol()
    , family = 'multinomial'
)

logReg_grid = (
    tune.ParamGridBuilder()
    .addGrid(logReg_obj.regParam
            , [0, 0.25, 0.5, .75, 1]
        )
    .addGrid(logReg_obj.elasticNetParam
            , [0, 0.25, 0.5, .75, 1]
        )
    .build()
)

logReg_ev = ev.MulticlassClassificationEvaluator(
    predictionCol = 'prediction'
    , labelCol = 'satisfied')

cross_v = tune.CrossValidator(
    estimator = logReg_obj
    , estimatorParamMaps = logReg_grid
    , evaluator = logReg_ev
)

pipeline = Pipeline(stages = [vectorAssembler, selector])
data_trans = pipeline.fit(airline_train)

logReg_modelTest = cross_v.fit(
    data_trans.transform(airline_train)
)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

An error was encountered:
Invalid status code '400' from http://localhost:8998/sessions/1/statements/28 with error payload: {"msg":"requirement failed: Session isn't active."}


Now that we have the best parameters for the model, we will apply the model to the test dataset and print the weighted precision, weighted recall, and accuracy of the model with the best parameters. We will also print what the best parameters are in this cell.

In [ ]:
data_trans_test = data_trans.transform(airline_test)
results = logReg_modelTest.transform(data_trans_test)

print(logReg_ev.evaluate(results, {logReg_ev.metricName: 'weightedPrecision'}))
print(logReg_ev.evaluate(results, {logReg_ev.metricName: 'weightedRecall'}))
print(logReg_ev.evaluate(results, {logReg_ev.metricName: 'accuracy'}))
print('Best params, regParam: %s, elasticNetParam: %s' 
      %(logReg_modelTest.bestModel._java_obj.getRegParam(),
      logReg_modelTest.bestModel._java_obj.getElasticNetParam()))

VBox()

An error was encountered:
Session 1 unexpectedly reached final status 'dead'. See logs:
stdout: 
#
# There is insufficient memory for the Java Runtime Environment to continue.
# Native memory allocation (mmap) failed to map 425680896 bytes for committing reserved memory.
# An error report file with more information is saved as:
# /tmp/hs_err_pid22193.log

stderr: 
21/11/29 07:17:44 INFO TaskSetManager: Finished task 3.0 in stage 334.0 (TID 1301) in 96 ms on ip-172-31-78-127.ec2.internal (executor 1) (2/4)
21/11/29 07:17:44 INFO TaskSetManager: Finished task 0.0 in stage 334.0 (TID 1298) in 107 ms on ip-172-31-78-127.ec2.internal (executor 1) (3/4)
21/11/29 07:17:44 INFO TaskSetManager: Finished task 1.0 in stage 334.0 (TID 1299) in 117 ms on ip-172-31-66-80.ec2.internal (executor 3) (4/4)
21/11/29 07:17:44 INFO DAGScheduler: ResultStage 334 (countByValue at ChiSqTest.scala:124) finished in 0.124 s
21/11/29 07:17:44 INFO DAGScheduler: Job 297 finished: countByValue at ChiSqTest.scala:12

Next, we will use GridSearch to search for the best parameters for the the random forest classifier. The same steps that were used for logistic regression will be applied for the random forest classifier.

In [ ]:
vectorAssembler = feat.VectorAssembler(
    inputCols = ordered_airline.columns[1:-1]
    , outputCol = 'features')

selector = feat.ChiSqSelector(
    labelCol = 'satisfied'
    , numTopFeatures = 5
    , outputCol = 'selected')

rf_obj = cl.RandomForestClassifier(
    labelCol = 'satisfied'
    , featuresCol = selector.getOutputCol()
    , minInstancesPerNode = 10
    , numTrees = 10
)

rf_grid = (
    tune.ParamGridBuilder()
    .addGrid(rf_obj.minInstancesPerNode
            , [5, 10, 15, 25, 50]
        )
    .addGrid(rf_obj.numTrees
            , [5, 10, 15, 25, 50]
        )
    .build()
)

rf_ev = ev.MulticlassClassificationEvaluator(
    predictionCol = 'prediction'
    , labelCol = 'satisfied')

cross_v = tune.CrossValidator(
    estimator = rf_obj
    , estimatorParamMaps = rf_grid
    , evaluator = rf_ev
)

pipeline = Pipeline(stages = [vectorAssembler, selector])
data_trans = pipeline.fit(airline_train)

rf_modelTest = cross_v.fit(
    data_trans.transform(airline_train)
)

VBox()

An error was encountered:
Session 1 unexpectedly reached final status 'dead'. See logs:
stdout: 
#
# There is insufficient memory for the Java Runtime Environment to continue.
# Native memory allocation (mmap) failed to map 425680896 bytes for committing reserved memory.
# An error report file with more information is saved as:
# /tmp/hs_err_pid22193.log

stderr: 
21/11/29 07:17:44 INFO TaskSetManager: Finished task 3.0 in stage 334.0 (TID 1301) in 96 ms on ip-172-31-78-127.ec2.internal (executor 1) (2/4)
21/11/29 07:17:44 INFO TaskSetManager: Finished task 0.0 in stage 334.0 (TID 1298) in 107 ms on ip-172-31-78-127.ec2.internal (executor 1) (3/4)
21/11/29 07:17:44 INFO TaskSetManager: Finished task 1.0 in stage 334.0 (TID 1299) in 117 ms on ip-172-31-66-80.ec2.internal (executor 3) (4/4)
21/11/29 07:17:44 INFO DAGScheduler: ResultStage 334 (countByValue at ChiSqTest.scala:124) finished in 0.124 s
21/11/29 07:17:44 INFO DAGScheduler: Job 297 finished: countByValue at ChiSqTest.scala:12

Lastly, we will display the performance of the random forest classifier model with the best parameters from GridSearch and display the best parameters chosen from GridSearch.

In [ ]:
data_trans_test = data_trans.transform(airline_test)
results = rf_modelTest.transform(data_trans_test)

print(rf_ev.evaluate(results, {rf_ev.metricName: 'weightedPrecision'}))
print(rf_ev.evaluate(results, {rf_ev.metricName: 'weightedRecall'}))
print(rf_ev.evaluate(results, {rf_ev.metricName: 'accuracy'}))
print('Best params, minInstancesPerNode: %s, numTrees: %s' 
      %(rf_modelTest.bestModel._java_obj.getminInstancesPerNode(),
      rf_modelTest.bestModel._java_obj.getnumTrees()))

VBox()

An error was encountered:
Session 1 unexpectedly reached final status 'dead'. See logs:
stdout: 
#
# There is insufficient memory for the Java Runtime Environment to continue.
# Native memory allocation (mmap) failed to map 425680896 bytes for committing reserved memory.
# An error report file with more information is saved as:
# /tmp/hs_err_pid22193.log

stderr: 
21/11/29 07:17:44 INFO TaskSetManager: Finished task 3.0 in stage 334.0 (TID 1301) in 96 ms on ip-172-31-78-127.ec2.internal (executor 1) (2/4)
21/11/29 07:17:44 INFO TaskSetManager: Finished task 0.0 in stage 334.0 (TID 1298) in 107 ms on ip-172-31-78-127.ec2.internal (executor 1) (3/4)
21/11/29 07:17:44 INFO TaskSetManager: Finished task 1.0 in stage 334.0 (TID 1299) in 117 ms on ip-172-31-66-80.ec2.internal (executor 3) (4/4)
21/11/29 07:17:44 INFO DAGScheduler: ResultStage 334 (countByValue at ChiSqTest.scala:124) finished in 0.124 s
21/11/29 07:17:44 INFO DAGScheduler: Job 297 finished: countByValue at ChiSqTest.scala:12